In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import api_keys
from spotipy.oauth2 import SpotifyOAuth
import os
import requests
from dotenv import load_dotenv
import base64
import json
import webbrowser
import urllib

## https://developer.spotify.com/documentation/web-api/reference/get-users-saved-tracks

In [4]:
load_dotenv()
REDIRECT_URI = "http://localhost"
CLIENT_ID = os.getenv("client_id")
CLIENT_SECRET = os.getenv("client_secret")
SCOPE = 'user-library-read'

## Request User Authorization

In [5]:
# Create the authorization URL
params = {
    'response_type': 'code',
    'client_id': CLIENT_ID,
    'scope': SCOPE,
    'redirect_uri': REDIRECT_URI,
}
url = f"https://accounts.spotify.com/authorize?{urllib.parse.urlencode(params)}"

# Open the authorization URL in the default web browser
webbrowser.open(url)

True

In [6]:
CODE = "AQB1mCqL-L8xCt3hampq-kMdBdGK4BCwrmq4t1nhhNcKxcT38SSvVJ9ucxLYms_81PgryUq7bF2Plwm_fs_wiKG8xoJtTAv0JXgCIhAwKIzzxemaJ6zz9jaGs-VuZO2vyfjZpwhFakT02bzuLXzumFaLXsbTSPMTfCYgEZ9PRuL_GrABOTLE"

## Get Token Info
- Lasts only 1 hour

In [8]:
def get_token_info():
    auth_header = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode('utf-8')).decode('utf-8')
    # auth_string = client_id + ":" + client_secret
    # auth_bytes = auth_string.encode("utf-8")
    # auth_base64 = str(base64.b64encode(auth_bytes), "utf-8" )

    url = "https://accounts.spotify.com/api/token"
    headers = {
            "Authorization": f"Basic {auth_header}",
            "Content-Type": "application/x-www-form-urlencoded",
    }
    data = {
            "grant_type": "authorization_code", 
            "code" : CODE, 
            "redirect_uri": REDIRECT_URI}
    result = requests.post(url, headers=headers, data=data)
    json_results = json.loads(result.content)
    token = json_results["access_token"]
    refresh_token = json_results["refresh_token"]
    return json_results

token_info = get_token_info()

In [9]:
token_info

{'access_token': 'BQDVM0uIACQHhxdAMHEncs6IyRT4Q0VpDkiffH24mbfWKKxLgykJObxI5hTuUzo6B_JzidUHVTo39ErRu5uDonKkhLwNO5_PTDTIA8TYMD4pbQOpfCStmrDs1vHhyXHAPnxE1Ks5tbrdDz2Pbxg6IiJRb8CFjIDsvsCt3DmqkjzNMDSYtXW7AdY8N96dFIU',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'refresh_token': 'AQCoVTURqGprWDpgZ-ubm2rwxImnf2qSiZi2x-OGnFWwj1Mz_q_Bj8fi25sJeeL99t0T1JJKPH8mwG63hxpTQHpPcvdYcthXIhPJ15w4atTfDtD1NIT_YqzIkV0x_o6Ljnc',
 'scope': 'user-library-read'}

In [10]:
token = token_info["access_token"]
refresh_token = token_info["refresh_token"]
scope = token_info["scope"]

## Getting Refresh Token

In [11]:
# def authorize():
#     # The scopes your application needs access to
#     scopes = ['user-library-read']

#     # Create the authorization URL
#     params = {
#         'client_id': client_id,
#         'response_type': 'code',
#         'redirect_uri': REDIRECT_URI,
#         'scope': ' '.join(scopes)

## Refresh Token

In [12]:

# client_id = os.getenv("client_id")
# client_secret = os.getenv("client_secret")

# def get_token():
#     auth_string = f"{client_id}:{client_secret}"
#     auth_bytes = auth_string.encode("utf-8")
#     auth_base64 = str(base64.b64encode(auth_bytes), "utf-8" )

#     url = "https://accounts.spotify.com/api/token"
#     headers = {
#             "Authorization": f"Basic {auth_base64}",
#             "Content-Type": "application/x-www-form-urlencoded"
#     }
#     data = {"grant_type": "refresh_token"}
#     result = post(url, headers=headers, data=data)
#     json_results = json.loads(result.content)
#     token = json_results["access_token"]
#     refresh_token = json_results["refresh_token"]
#     return token, refresh_token

# refresh_token = get_token()

## Get Auth Header

In [13]:
def get_auth_header(token):
    return f"Bearer " + token

In [14]:
get_auth_header(token)

'Bearer BQDVM0uIACQHhxdAMHEncs6IyRT4Q0VpDkiffH24mbfWKKxLgykJObxI5hTuUzo6B_JzidUHVTo39ErRu5uDonKkhLwNO5_PTDTIA8TYMD4pbQOpfCStmrDs1vHhyXHAPnxE1Ks5tbrdDz2Pbxg6IiJRb8CFjIDsvsCt3DmqkjzNMDSYtXW7AdY8N96dFIU'

## Post Request
- Request using token for API endpoints

In [15]:
SPOTIFY_GET_CURRENT_USER_SAVED_TRACKS_URL = "https://api.spotify.com/v1/me/tracks?limit=50&offset=0"
SPOTIFY_GET_CURRENT_TRACK_URL = 'https://api.spotify.com/v1/me/player/currently-playing'

In [16]:
def get_current_user_tracks(url):
    response = requests.get(url=url, headers={"Authorization" : f"{get_auth_header(token)}"})
    resp_json = response.json()
    return resp_json

In [17]:
def loop_through():
    first_spotify_url = "https://api.spotify.com/v1/me/tracks?limit=50&offset=0"
    results = get_current_user_tracks(first_spotify_url)
    # increment_of50_list = [i for i in range(0, get_current_user_tracks()["total"], 50)]
    # increment_of50_list_test = [i for i in range(0, int(get_current_user_tracks(SPOTIFY_GET_CURRENT_USER_SAVED_TRACKS_URL)["total"]) - 4300, 50)]
    full_list = []
    current_url = results["href"]
    # while results["next"] is not None:
    for i in range(4):
        results = get_current_user_tracks(current_url)
        full_list.append(results)
        spotify_url_next = results["next"]
        current_url = spotify_url_next
    return full_list
            

In [18]:
test_results = loop_through()

## Results

In [21]:
full_list_parsed = []
for index_i, i in enumerate(test_results):  
    for index_j, j in enumerate(test_results[index_i]["items"]):  
        sub_list = []
        tracks_details_all = {}
        track_details = {}
        added_to_playlist_time =  "added_to_playlist_time: "+j["added_at"]
        artist_name =  "artist_name: "+j["track"]["album"]["artists"][0]["name"]
        unique_artist_id =  "unique_artist_id: "+j["track"]["album"]["artists"][0]["uri"]
        release_date =  "release_date: "+j["track"]["album"]["release_date"]
        song_name =  "song_name: "+j["track"]["name"]
        song_length_seconds = "song_length_seconds: "+str(int(j["track"]["duration_ms"])/1000)
        song_explicit = "song_explicit: "+str(j["track"]["explicit"])
        song_linkId = "song_linkId: "+j["track"]["external_urls"]["spotify"]
        song_popularity = "song_popularity: "+str(j["track"]["popularity"])
        album_name = "album_name: "+j["track"]["album"]["name"]
        spotify_uri_track = "spotify_uri_track: "+j["track"]["album"]["uri"]
        track_details[f"{index_i}.{index_j}"] = [added_to_playlist_time, artist_name, unique_artist_id, release_date, song_name, song_length_seconds, 
                                                    song_explicit, song_linkId, song_popularity, album_name, spotify_uri_track]
        full_list_parsed.append(track_details)
print(full_list_parsed[0:5])
        # print(index_i, j)

[{'0.0': ['added_to_playlist_time: 2023-06-09T15:26:19Z', 'artist_name: RL Grime', 'unique_artist_id: spotify:artist:5eIbEEQnDM8yuDVB0bimSP', 'release_date: 2023-06-09', 'song_name: Pour Your Heart Out (feat. 070 Shake)', 'song_length_seconds: 223.561', 'song_explicit: False', 'song_linkId: https://open.spotify.com/track/4p16E9c9Ig6xFMGS3Y82mT', 'song_popularity: 66', 'album_name: Pour Your Heart Out (feat. 070 Shake)', 'spotify_uri_track: spotify:album:3IqciDNeCpyycdfxs6AA6T']}, {'0.1': ['added_to_playlist_time: 2023-06-08T15:48:43Z', 'artist_name: NURKO', 'unique_artist_id: spotify:artist:757FXqX0Osk2pqtgv4E5v4', 'release_date: 2019-01-31', 'song_name: Breathe Without', 'song_length_seconds: 226.315', 'song_explicit: False', 'song_linkId: https://open.spotify.com/track/3tAzIzJ76pdgn17SIoMSXO', 'song_popularity: 50', 'album_name: Breathe Without', 'spotify_uri_track: spotify:album:1b5bzXZrnWkcY3llwsRvwP']}, {'0.2': ['added_to_playlist_time: 2023-06-05T18:57:37Z', 'artist_name: NURKO',

In [27]:
[i for i in full_list_parsed for j in i]


[{'0.0': ['added_to_playlist_time: 2023-06-09T15:26:19Z',
   'artist_name: RL Grime',
   'unique_artist_id: spotify:artist:5eIbEEQnDM8yuDVB0bimSP',
   'release_date: 2023-06-09',
   'song_name: Pour Your Heart Out (feat. 070 Shake)',
   'song_length_seconds: 223.561',
   'song_explicit: False',
   'song_linkId: https://open.spotify.com/track/4p16E9c9Ig6xFMGS3Y82mT',
   'song_popularity: 66',
   'album_name: Pour Your Heart Out (feat. 070 Shake)',
   'spotify_uri_track: spotify:album:3IqciDNeCpyycdfxs6AA6T']},
 {'0.1': ['added_to_playlist_time: 2023-06-08T15:48:43Z',
   'artist_name: NURKO',
   'unique_artist_id: spotify:artist:757FXqX0Osk2pqtgv4E5v4',
   'release_date: 2019-01-31',
   'song_name: Breathe Without',
   'song_length_seconds: 226.315',
   'song_explicit: False',
   'song_linkId: https://open.spotify.com/track/3tAzIzJ76pdgn17SIoMSXO',
   'song_popularity: 50',
   'album_name: Breathe Without',
   'spotify_uri_track: spotify:album:1b5bzXZrnWkcY3llwsRvwP']},
 {'0.2': ['added_

In [31]:
for i in full_list_parsed:
    print(i)

{'0.0': ['added_to_playlist_time: 2023-06-09T15:26:19Z', 'artist_name: RL Grime', 'unique_artist_id: spotify:artist:5eIbEEQnDM8yuDVB0bimSP', 'release_date: 2023-06-09', 'song_name: Pour Your Heart Out (feat. 070 Shake)', 'song_length_seconds: 223.561', 'song_explicit: False', 'song_linkId: https://open.spotify.com/track/4p16E9c9Ig6xFMGS3Y82mT', 'song_popularity: 66', 'album_name: Pour Your Heart Out (feat. 070 Shake)', 'spotify_uri_track: spotify:album:3IqciDNeCpyycdfxs6AA6T']}
{'0.1': ['added_to_playlist_time: 2023-06-08T15:48:43Z', 'artist_name: NURKO', 'unique_artist_id: spotify:artist:757FXqX0Osk2pqtgv4E5v4', 'release_date: 2019-01-31', 'song_name: Breathe Without', 'song_length_seconds: 226.315', 'song_explicit: False', 'song_linkId: https://open.spotify.com/track/3tAzIzJ76pdgn17SIoMSXO', 'song_popularity: 50', 'album_name: Breathe Without', 'spotify_uri_track: spotify:album:1b5bzXZrnWkcY3llwsRvwP']}
{'0.2': ['added_to_playlist_time: 2023-06-05T18:57:37Z', 'artist_name: NURKO', 'u